#### Этот ноутбук реализует:

1. Фильтрацию высоких частот ГИС сигнала оконным фильтром
2. Интерполяцию данных ГИС на сетку с шагом 4 метра.

#### Данный ноутбук нужен для:
* Логирования интерполированных ГИС данных в excel_file_path.xlsx

#### Замечание:
Данный ноутбук - плод творческих изысканий Андрея Сорокина. Код запускался определенными кусками через Spyder.

In [ ]:
import numpy as np
from scipy.signal import convolve
import matplotlib.pyplot as plt
import pandas as pd
from scipy import interpolate 
# Функция для применения низкочастотного фильтра с использованием свертки
def apply_low_pass_filter(signal, cutoff_frequency, sample_rate):
    """
    Применяет низкочастотный фильтр, используя свертку с функцией sinc.

    :param signal: Входной сигнал.
    :param cutoff_frequency: Частота среза низкочастотного фильтра.
    :param sample_rate: Частота дискретизации сигнала.

    :return: Отфильтрованный сигнал.
    """
    # Определяем длину фильтра
    filter_length = int(sample_rate / cutoff_frequency) * 10

    # Убеждаемся, что длина фильтра нечетная
    if filter_length % 2 == 0:
        filter_length += 1

    # Генерируем временной массив
    t = np.arange(-filter_length // 2, filter_length // 2 + 1)

    # Создаем фильтр sinc
    sinc_filter = np.sinc(2 * cutoff_frequency * t / sample_rate)

    # Применяем оконную функцию к фильтру sinc для уменьшения осцилляций Гиббса
    window = np.hamming(len(sinc_filter))
    sinc_filter *= window

    # Нормализуем фильтр
    sinc_filter /= np.sum(sinc_filter)

    # Применяем фильтр к сигналу с помощью свертки
    filtered_signal = convolve(signal, sinc_filter, mode='same')

    return filtered_signal

# Загружаем данные из файлов
data = pd.read_excel('Wells_carot_SP.xlsx')
wells_data = pd.read_excel('Wells_info.xlsx')
index_seism= pd.read_excel('Wells_index_seism.xlsx')
# Сопоставляем значения скорости (v) из Wells_info с колонками в Proc_signal_BK
v_values = wells_data.set_index('Well')['v'].to_dict()

# Приблизительно определяем частоту дискретизации (исходя из предположения, что глубина равномерно дискретизирована)
depths = data.iloc[:, 0]
sample_rate = 1 / (depths[1] - depths[0])  # Предполагаем равномерные интервалы по глубине

# Применяем низкочастотный фильтр к данным первых трех скважин для демонстрации
filtered_data = data.copy()  # Создаем копию для хранения отфильтрованных данных
result_array = np.zeros((1901, 134))
# Фильтруем и строим графики для первых трех скважин
# Передискретизация значений глубины для каждой скважины
filtered_data_array = []

for column in data.columns[1:4]:
    if column in v_values:
        v = v_values[column]
        new_depths = np.arange(depths.min(), depths.max(), 4)
         
        # Интерполируем исходные данные для новых значений глубины
        interpolated_data = pd.DataFrame({'Depth': new_depths})
        interpolated_data = interpolated_data.set_index('Depth')
        interpolated_data[column] = np.interp(new_depths, depths, data[column])

        sample_rate = 1 / (new_depths[1] - new_depths[0])  # Пересчитываем частоту дискретизации для новых значений глубины

        cutoff_frequency = 80 / v
        signal = interpolated_data[column].fillna(0)  # Заменяем NaN на 0 для фильтрации
        filtered_signal = apply_low_pass_filter(signal, cutoff_frequency, sample_rate)

        # Строим графики исходных и отфильтрованных сигналов для сравнения
        plt.figure(figsize=(10, 6))

        plt.plot(new_depths, signal, label='Original Signal')
        plt.plot(new_depths, filtered_signal, label='Filtered Signal')
        plt.title(f'Сигналы для скважины: {column}')
        plt.xlabel('Глубина')
        plt.xlim(2500, 3000)
        plt.ylabel('Амплитуда')
        plt.legend()
        plt.show()
        
        
#         filtered_data_array.append(np.column_stack((new_depths, filtered_signal)))
        
# filtered_data_array = np.array(filtered_data_array)

result_array = np.zeros((len(new_depths) + 1, len(data.columns[1:134]) + 1), dtype=object)


# Создаем массив с названиями столбцов
column_names = ['Depth'] + list(data.columns[1:146])

# Создаем пустой массив result_array с нужными размерами и типом данных
result_array = np.zeros((len(new_depths) + 1, len(data.columns[1:146]) + 1), dtype=object)

# Записываем названия столбцов в первую строку
result_array[0] = column_names

for i, column in enumerate(data.columns[1:146]):
    if column in v_values:
        v = v_values[column]
        new_depths = np.arange(depths.min(), depths.max(), 4)
        
        # Интерполируем исходные данные для новых значений глубины
        interpolated_data = pd.DataFrame({'Depth': new_depths})
        interpolated_data = interpolated_data.set_index('Depth')
        interpolated_data[column] = np.interp(new_depths, depths, data[column])

        sample_rate = 1 / (new_depths[1] - new_depths[0])  # Пересчитываем частоту дискретизации для новых значений глубины

        cutoff_frequency = 80 / v
        signal = interpolated_data[column].fillna(0)  # Заменяем NaN на 0 для фильтрации
        filtered_signal = apply_low_pass_filter(signal, cutoff_frequency, sample_rate)

        # Сохраняем отфильтрованный сигнал в массив результатов
        result_array[1:, i+1] = filtered_signal

# Добавляем значения new_depths в первый столбец
result_array[1:, 0] = new_depths


# Создаем DataFrame из массива result_array
df = pd.DataFrame(result_array[1:], columns=result_array[0])

# Указываем путь для сохранения файла Excel
excel_file_path = 'filtered2_data.xlsx'

# Сохраняем DataFrame в файл Excel
df.to_excel(excel_file_path, index=False)

print(f"Результаты успешно сохранены в файл: {excel_file_path}")


